In [21]:
# general imports
import pandas as pd
import requests
import json

In [6]:
# load in files
keywords_df = pd.read_csv('data/keywords.csv')
movies_metadata_df = pd.read_csv('data/movies_metadata.csv')
ratings_df = pd.read_csv('data/ratings.csv')
bechdel_scores = 

/var/folders/lp/wvqcc0x13jjb99jsy0mqncqr0000gn/T/ipykernel_59991/435099851.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('data/movies_metadata.csv')


In [7]:
# general cleaning

# dropping unecessary columns from movies_metadata
movies_metadata_df = movies_metadata_df.drop(columns = ['belongs_to_collection', 'homepage', 
                                                        'poster_path', 'status', 'tagline', 
                                                        'video', 'spoken_languages', 'vote_average', 
                                                        'vote_count'])
movies_metadata_df['id'] = pd.to_numeric(movies_metadata_df['id'] , downcast='integer', errors='coerce')

# get average rating by 'movieId', drop unecessary columns, rename 'movieId'
avg_rating = ratings_df.groupby('movieId').mean().reset_index()
avg_rating = avg_rating.drop(columns = ['userId', 'timestamp'])
avg_rating = avg_rating.rename(columns = {'movieId': 'id'})

In [8]:
# joining movies_metadata_df and keywords_df together
joined_df = pd.merge(movies_metadata_df, keywords_df, on='id', how='inner')
# drop rows where 'id' or 'imdb_id' is NaN
joined_df = joined_df.dropna(subset=['id', 'imdb_id'])

In [9]:
# function to clean imdb_id of leading 'tt' for API calls
def clean_imbd(imdb_ids):
    return imdb_ids.replace('t', '')

# apply function to imdb_id column 
joined_df['imdb_id'] = joined_df['imdb_id'].apply(clean_imbd)

In [10]:
joined_df.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,title,keywords
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Toy Story,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Jumanji,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,Grumpier Old Men,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,Waiting to Exhale,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862.0,0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,Father of the Bride Part II,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [18]:
## Bechdel Test API Call - leave commented to avoid rerunning unecessarily 
## resulting .json can be found in data folder: 'data/bechdel_scores.json'

# url = "http://bechdeltest.com/api/v1/getAllMovies"
# response = requests.get(url)
# bechdel_scores = response.json()
# file_path = "bechdel_scores.json"

# with open(file_path, 'w') as json_file:
#     json.dump(bechdel_scores, json_file, indent=4)

# print(f"JSON data written to {file_path}")

In [41]:
# create a pandas DataFrame with imdb_id and respective bechdel_score
imdb_bechdel = {}
for movie in bechdel_scores:
    imdb_bechdel[movie['imdbid']] = movie['rating']
bechdel_df = pd.DataFrame()
bechdel_df['imdb_id'] = imdb_bechdel.keys()
bechdel_df['bechdel_score'] = imdb_bechdel.values()
bechdel_df.head()

,imdb_id,bechdel_score
0,0267536,3
1,3155794,0
2,14495706,0
3,2221420,0
4,12592084,0


In [36]:
# merge bechdel with movie dataset
bechdel_movies = pd.merge(joined_df, bechdel_df, on='imdb_id', how='inner')
# convert release_date to datetime object
bechdel_movies['release_date'] = pd.to_datetime(bechdel_movies['release_date'])

In [37]:
bechdel_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8069 entries, 0 to 8068
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   adult                 8069 non-null   object        
 1   budget                8069 non-null   object        
 2   genres                8069 non-null   object        
 3   id                    8069 non-null   float64       
 4   imdb_id               8069 non-null   object        
 5   original_language     8068 non-null   object        
 6   original_title        8069 non-null   object        
 7   overview              8060 non-null   object        
 8   popularity            8069 non-null   object        
 9   production_companies  8069 non-null   object        
 10  production_countries  8069 non-null   object        
 11  release_date          8068 non-null   datetime64[ns]
 12  revenue               8069 non-null   float64       
 13  runtime           

In [38]:
## Write DataFrame to .csv file - Leave commented
## resulting .csv can be found in data folder: 'data/bechdel_movies.csv'

#bechdel_movies.to_csv('bechdel_movies.csv', index=False, sep=',', na_rep='', encoding='utf-8')